<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task1/blob/main/DL_BETO_Task1_en_23_8_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.3 MB/s eta 0:00:00


In [2]:
#Leer los archivos TSV y seleccionar las columnas deseadas y filtrar por idioma 'es'

import pandas as pd
from sklearn.utils import shuffle
test_data = pd.read_csv('/content/EXIST2021_test_labeled.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'es']
train_data = pd.read_csv('/content/EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'es']
#train_data_en = pd.read_csv('/content/train_data_translated_en.csv', sep=',', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'en']
swapp = pd.read_csv('/content/swapp_EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'es']

In [3]:
# Concatenar train_data y train_data_en en un único DataFrame llamado train_data
train_data = pd.concat([train_data, swapp], ignore_index=True)
train_data = shuffle(train_data, random_state=42)

# Verificar el resultado
print(train_data.head())

     language                                               text       task1
9224       es  M_OlgaSCordero @ Lopezobrador_ @ Pura Florero ...      sexist
3464       es  Si @auronplay llega a ver esto tiro mi telefon...  non-sexist
6984       es  @jjmiphoto Man...se lo dije a mi esposo ayer y...  non-sexist
4132       es  Una vieja me llama hace como un año erróneamen...  non-sexist
4955       es                            Lleva falda, muy corta.      sexist


In [4]:
columna_clase_objetivo = "task1"

df = train_data

# Contar la cantidad de categorías únicas en la columna de la clase objetivo
cantidad_categorias = df[columna_clase_objetivo].nunique()

# Contar la cantidad de muestras para cada categoría de la clase objetivo
muestras_por_categoria = df[columna_clase_objetivo].value_counts()

# Imprimir los resultados
print("Cantidad de categorías:", cantidad_categorias)
print("Muestras por categoría:")
print(muestras_por_categoria)
df.shape[0]

Cantidad de categorías: 2
Muestras por categoría:
non-sexist    4876
sexist        4706
Name: task1, dtype: int64


9582

#**PREPROCESADO**


---



In [ ]:
import re

def anonimize_text(text):
    # Anonimizar nombres de usuario
    text = re.sub(r'@[\w_]+', '@user', text)

    # Anonimizar hashtags
    text = re.sub(r'#\w+', '#hashtag', text)

    # Anonimizar enlaces
    text = re.sub(r'https?://\S+', 'link', text)

    return text

train_data['text'] = train_data['text'].apply(anonimize_text)
test_data['text'] = test_data['text'].apply(anonimize_text)

In [ ]:
# Convertir a minúsculas

train_data.text = train_data.text.str.lower()
train_data.head(10)
test_data.text = test_data.text.str.lower()
test_data.head(10)

,language,text,task1
0,en,pennsylvania state rep horrifies with opening ...,non-sexist
1,en,"@user he sounds like as ass, and very condesce...",non-sexist
2,en,"@user @user lol! ""this behavior of not letting...",sexist
3,en,@user @user rights?i mean yeah most women espe...,sexist
4,en,the jack manifold appreciation i’m seeing is o...,non-sexist
5,en,@user why isn't this character taking a lie de...,non-sexist
6,en,@user i can’t help but torture myself reading ...,non-sexist
7,en,@user @user @user stop regarding women as anim...,sexist
8,en,"yeah, it is rough, but not for women. married ...",sexist
9,en,although the christian denial isn't emphasized...,non-sexist


In [ ]:
# Eliminar caráctres especiales, #:)(/\='] usados en emojis y hashtags

train_data.text = train_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
train_data.head(10)
test_data.text = test_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
test_data.head(10)

<ipython-input-7-091c5808df50>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data.text = train_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
<ipython-input-7-091c5808df50>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data.text = test_data.text.str.replace("[^a-zA-ZñÑ#]", " ")


,language,text,task1
0,en,pennsylvania state rep horrifies with opening ...,non-sexist
1,en,user he sounds like as ass and very condesce...,non-sexist
2,en,user user lol this behavior of not letting...,sexist
3,en,user user rights i mean yeah most women espe...,sexist
4,en,the jack manifold appreciation i m seeing is o...,non-sexist
5,en,user why isn t this character taking a lie de...,non-sexist
6,en,user i can t help but torture myself reading ...,non-sexist
7,en,user user user stop regarding women as anim...,sexist
8,en,yeah it is rough but not for women married ...,sexist
9,en,although the christian denial isn t emphasized...,non-sexist




---



In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# Crear una instancia de LabelEncoder
SEED = 4
le = LabelEncoder()
le.fit(train_data['task1'])

# Preprocess the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(list(train_data['text']), list(le.transform(train_data['task1'])),
                                                                    test_size=.2, random_state=SEED, shuffle=True)

In [10]:
train_data.head(10)

,language,text,task1
9224,es,M_OlgaSCordero @ Lopezobrador_ @ Pura Florero ...,0
3464,es,Si @auronplay llega a ver esto tiro mi telefon...,1
6984,es,@jjmiphoto Man...se lo dije a mi esposo ayer y...,1
4132,es,Una vieja me llama hace como un año erróneamen...,1
4955,es,"Lleva falda, muy corta.",0
7709,es,"Paulitacarasa @ ... , pobre con tanta mujer al...",0
2663,es,Por violar a su hijastra de 14 años de edad un...,0
8078,es,LO MEJOR Y LO MÁS LEÍDO DEL 2020 El nacimiento...,1
7638,es,"SenadorLuenzo @ No podes ser tan , payaso salu...",1
3686,es,@carlosjose0968 Si pidió opiniones y tu opinas...,1


In [6]:
# Preprocess the dataset
test_texts = list(test_data['text'])

test_labels = le.transform(test_data['task1'])


In [7]:
print(len(train_texts))
print(len(val_texts))
print(len(test_texts))

7665
1917
2160


In [9]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
# Define the model
model = "dccuchile/bert-base-spanish-wwm-cased"



In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=130)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=130)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=130)


In [11]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels
test_encodings['labels'] = test_labels

In [12]:
batch_size = 16
num_epochs = 4

In [13]:
!pip install datasets

from transformers import DataCollatorWithPadding
from datasets import Dataset

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [14]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [15]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler)
loss=SparseCategoricalCrossentropy(from_logits=True)

In [16]:
model = TFAutoModelForSequenceClassification.from_pretrained(model, num_labels=2,
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.1,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:


model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
)


In [18]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/4
480/480 [==============================] - 301s 507ms/step - loss: 0.4407 - accuracy: 0.7932 - val_loss: 0.2545 - val_accuracy: 0.9025
Epoch 2/4
480/480 [==============================] - 235s 489ms/step - loss: 0.1212 - accuracy: 0.9611 - val_loss: 0.1391 - val_accuracy: 0.9619
Epoch 3/4
480/480 [==============================] - 234s 486ms/step - loss: 0.0335 - accuracy: 0.9896 - val_loss: 0.1149 - val_accuracy: 0.9718
Epoch 4/4
480/480 [==============================] - 231s 482ms/step - loss: 0.0103 - accuracy: 0.9971 - val_loss: 0.0907 - val_accuracy: 0.9797


In [19]:
tf_test_dataset = Dataset.from_dict(test_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,  # No necesitas barajar para predicción
    collate_fn=data_collator,
    batch_size=batch_size,
)


In [21]:
import numpy as np
preds = model.predict(tf_test_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)

135/135 [==============================] - 21s 158ms/step


In [22]:
from sklearn.metrics import classification_report
print(classification_report(class_preds, test_encodings['labels']))

              precision    recall  f1-score   support

           0       0.83      0.74      0.78      1164
           1       0.73      0.82      0.77       996

    accuracy                           0.78      2160
   macro avg       0.78      0.78      0.78      2160
weighted avg       0.78      0.78      0.78      2160



# **Resultados**

**Se utiliza el modelo BETO para tuits en español. Se utilizan los datos originales más un aumento de datos que proveinen de la técnica swapping.**
batch_size = 16
num_epochs = 4

#Train


```
Epoch 1/4
480/480 [==============================] - 301s 507ms/step - loss: 0.4407 - accuracy: 0.7932 - val_loss: 0.2545 - val_accuracy: 0.9025
Epoch 2/4
480/480 [==============================] - 235s 489ms/step - loss: 0.1212 - accuracy: 0.9611 - val_loss: 0.1391 - val_accuracy: 0.9619
Epoch 3/4
480/480 [==============================] - 234s 486ms/step - loss: 0.0335 - accuracy: 0.9896 - val_loss: 0.1149 - val_accuracy: 0.9718
Epoch 4/4
480/480 [==============================] - 231s 482ms/step - loss: 0.0103 - accuracy: 0.9971 - val_loss: 0.0907 - val_accuracy: 0.9797
```

#Evaluación


```
precision    recall  f1-score   support

           0       0.83      0.74      0.78      1164
           1       0.73      0.82      0.77       996

    accuracy                           0.78      2160
   macro avg       0.78      0.78      0.78      2160
weighted avg       0.78      0.78      0.78      2160
```

